<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/ColabQwenServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#script works on T4 only

#apparently not needed
!pip install flask-ngrok
!pip install flask
!pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate
#apparently not needed

In [1]:
#those required
#!pip install pyngrok==4.1.1
!pip install pyngrok
!pip install qwen-vl-utils[decord]
!pip install autoawq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
  Created wheel for autoawq: filename=autoawq-0.2.8-py3-none-any.whl size=108744 sha256=639cbedcc6d13775451e99db17b2c92975149eb337a56d73d03dd1fb0eb9fb67
  Stored in directory: /root/.cache/pip/wheels/fd/03/fe/99c1c678bfe8aca712186466969ed866

In [2]:
import requests
from io import BytesIO
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct-AWQ", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct-AWQ")




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.


model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [3]:
def process_message_single(prompt, question, path_to_video):
  messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "file:///content/" + path_to_video, #sample_data/5fps_10s_quarter_video_example.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": question }, #"Is there person visible. Answer with 1 word: yes or no."},
        ],
    }
  ]
  print(messages)
  # Preparation for inference
  text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
  )
  image_inputs, video_inputs = process_vision_info(messages)
  inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
  )
  inputs = inputs.to("cuda")

  # Inference
  generated_ids = model.generate(**inputs, max_new_tokens=128)
  generated_ids_trimmed = [
      out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
  ]
  output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
  )
  #print(output_text)
  return output_text



In [ ]:


from pyngrok import ngrok
from flask import Flask, request, Response
import os

# token got to https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_token = "yyyyy"

port = 5000
ngrok.set_auth_token(ngrok_token)

app = Flask(__name__)

is_busy = False

@app.route("/")
def root():
  return "Hello world colab pyngrok!"

@app.route("/req1", methods=['POST'])
def req1():
  global is_busy
  try:
    if 'video' not in request.files or 'text' not in request.form:
        return "Missing video or text data", 400

    video_file = request.files['video']
    text_data = request.form['text']
    if is_busy:
      print("Server is busy, skipping 10s chunk processing")
      return "Server is busy", 503
    is_busy = True

    # Print the received data
    #print(f"Received text data: {text_data}")
    #print(f"Received video file: {video_file.filename}")

    # You can process the video file here, e.g., save it to disk
    videoPath = "sample_data/"+video_file.filename
    video_file.save(videoPath)
    response = process_message_single("", text_data, videoPath)
    # Display the video in the notebook (not possible with ngrok)
    # display(HTML(f'<video controls><source src="data:video/mp4;base64,{base64.b64encode(video_file.read()).decode()}" type="video/mp4"></video>'))

    os.remove(videoPath)
    # Send the text data back as response
    print( str(response))
    is_busy = False
    return response
  except Exception as e:
        print(f"An error occurred: {e}")  # Log or handle the exception as needed
        is_busy = False
        return "Internal Server Error", 500  # Return a generic error response



if __name__ == "__main__":
  try:
    app.logger.info('Hello from Flask logger!')
    public_url = ngrok.connect(port).public_url
    print(public_url)
    app.logger.info(public_url)
    app.run(port=port)
  finally:
    ngrok.disconnect(public_url=public_url)

